In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../secretary')

import pandas as pd
from secretary import todo
import secretary.secretary_bot as sb
from io import StringIO


In [ ]:
response = '''{
    "6":{
        "status":"*complete*",
        "tags":[
            "food preparation"
        ],
        "type":"action_item",
        "due date":"2024-02-22",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Wrap the sandwiches for tomorrow",
        "notes":"Jack needs to wrap the sandwiches for tomorrow. *Sandwiches have been made.*"
    }
}'''

df = pd.read_json(StringIO(response), orient='index')
updated_tasks_indexes = df.index.values.tolist()
updated_tasks_indexes
df.loc[updated_tasks_indexes[0]]

In [ ]:
message_content = "I love peanuts and applejacks."
message = "From: Jack\n" + message_content

df = sb.extract_tasks(message)
df

In [ ]:
message_content = "Silvia's action item is to buy eggs by 10/03/2022."
message = "From: Jack\n" + message_content

df = sb.extract_tasks(message)
df

In [ ]:
with open('legalese_email.txt', 'r') as file:
    content = file.read()

message = "From: Jack\n" + content

df_legal = sb.extract_tasks(message)
df_legal

In [ ]:
todos = todo.Todo()

tasks = [
  {
    "tags": ["shopping", "shoes"],
    "type": "action_item",
    "due date": "2024-04-14",
    "requestor": "Jack",
    "actor": "Jack",
    "summary": "Buy shoes",
    "notes": "I need to buy some cool new kicks."
  },
  {
    "tags": ["shopping", "groceries"],
    "type": "action_item",
    "due date": "2024-05-14",
    "requestor": "Jack",
    "actor": "Silvia",
    "summary": "Buy eggs",
    "notes": "Silvia needs to buy eggs"
  }
]

df = pd.concat([df_legal, pd.DataFrame(tasks)], ignore_index=True)
todos.add_new_task_to_database(df)
todos.df

In [ ]:
comment = "From: Jack\nSilvia and I should call Wolfgang."

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nI bought some boots.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nI finished the task due on 04/14'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nActually, Nick should buy the eggs.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nI just love zebras!'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nI need to hang out with Kodak soon.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nMerge tasks 2 and 3 about the probate process estate accounts.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

In [ ]:
comment = 'From: Jack\nHow many probate tasks do I have open?'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

answers = sb.answer_questions_about_tasks(comment, similar_tasks_json)
print(comment)
print(answers)

In [ ]:
comment = 'From: Jack\nWhat probate tasks do I have open?'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

answers = sb.answer_questions_about_tasks(comment, similar_tasks_json)
print(comment)
print(answers)

In [ ]:
comment = 'From: Jack\nList the probate tasks I have open.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

answers = sb.answer_questions_about_tasks(comment, similar_tasks_json)
print(comment)
print(answers)

In [ ]:
comment = 'From: Jack\nI should ask Nick where Truist bank is located.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

answers = sb.answer_questions_about_tasks(comment, similar_tasks_json)
print(comment)
print(answers)

In [ ]:
comment = 'From: Jack\nWhere is Truist Bank based?'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

answers = sb.answer_questions_about_tasks(comment, similar_tasks_json)
print(comment)
print(answers)

In [ ]:
todos = todo.Todo('data/tasks_database')
todos.df.drop(columns='embedding')